In [ ]:
import os
import sys
PROJECT_ROOT = os.path.dirname(os.path.abspath(""))

if PROJECT_ROOT not in sys.path:
    sys.path.insert(0, PROJECT_ROOT)
from scripts.extract import WorldBankExtractOperator
from scripts.transform import SparkTransformOperator
from scripts.loads import DatabaseLoaderService
from scripts.load_wbapi_etl_config import ETLPipelineConfig
